Лабораторная #4. Сравнить различные подходы к векторизации и построению индексных структур на данных, загруженных в лабораторной #1. Свести все в одну таблицу результатов.

In [35]:
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [36]:
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

# Чтение текста страниц из файла content.txt
def read_content_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    pages = content.split('---page---')
    return [page.strip() for page in pages if page.strip()]

# Чтение мета данных из файла meta_info.txt
def read_meta_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        meta = file.read()
    return meta

path = r'/Users/vlad-zharkov/Documents/Учеба/inf_search/‹ ¡  1/scraped_data_3/www_sports_ru/content_3.txt'
path_processed = r'/Users/vlad-zharkov/Documents/Учеба/inf_search/‹ ¡  2/processed_content_3.txt'
content_pages = read_content_file(path_processed)

meta = read_meta_file(r"/Users/vlad-zharkov/Documents/Учеба/inf_search/‹ ¡  1/scraped_data_3/meta_info_www_sports_ru_3.txt")

# Парсинг мета данных
def parse_meta(meta):
    meta_data = []
    url_pattern = re.compile(r'URL: (.+)')
    text_length_pattern = re.compile(r'Text Length: (\d+)')
    meta_lines = meta.strip().split('\n')
    
    current_url = None
    for line in meta_lines:
        url_match = url_pattern.match(line)
        text_length_match = text_length_pattern.match(line)
        
        if url_match:
            current_url = url_match.group(1)
        elif text_length_match:
            text_length = int(text_length_match.group(1))
            meta_data.append({
                "url": current_url,
                "text_length": text_length
            })
    
    return meta_data

meta_data = parse_meta(meta)

data = []
for idx, doc in enumerate(meta_data):
    #print(content_pages[idx])
    data.append({
        "id": idx + 1,
        "url": doc["url"],
        "content": content_pages[idx]
    })



In [37]:
import re
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [doc['content'] for doc in data]

In [38]:
# Прямой индекс
def create_forward_index(data):
    forward_index = {}
    for doc in data:
        forward_index[doc['id']] = doc['content']
    return forward_index

forward_index = create_forward_index(data)

# Функция для поиска в прямом индексе
def search_forward_index(query, forward_index):
    query_words = re.findall(r'\w+', query.lower())
    doc_scores = defaultdict(int)
    for doc_id, content in forward_index.items():
        for word in query_words:
            if word in content.lower():
                doc_scores[doc_id] += 1
    sorted_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    return [doc_id for doc_id, _ in sorted_docs]

In [39]:
# Обратный индекс
def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    for doc_id, content in enumerate(documents):
        words = re.findall(r'\w+', content.lower())
        for word in words:
            inverted_index[word].append(doc_id)
    return inverted_index

inverted_index = create_inverted_index(documents)

# Функция для поиска в обратном индексе
def search_inverted_index(query, index):
    query_words = re.findall(r'\w+', query.lower())
    doc_scores = defaultdict(int)
    for word in query_words:
        if word in index:
            for doc_id in index[word]:
                doc_scores[doc_id] += 1
    sorted_docs = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    return [doc_id for doc_id, _ in sorted_docs]

In [40]:
# TF-IDF индекс
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Функция для поиска в TF-IDF индексе
def search_tfidf_index(query, tfidf_matrix, vectorizer):
    query_vec = vectorizer.transform([query])
    cosine_similarities = (tfidf_matrix * query_vec.T).toarray().flatten()
    ranked_results = np.argsort(cosine_similarities)[::-1]
    return ranked_results, cosine_similarities

In [44]:
from sklearn.metrics import average_precision_score, ndcg_score
import numpy as np

def calculate_metrics(true_relevance, scores, k=5):
    true_relevance = np.asarray(true_relevance)
    scores = np.asarray(scores)
    
    # AP@5
    ap_5 = average_precision_score(true_relevance[:k], scores[:k])
    
    # MAP@5
    map_5 = np.mean([average_precision_score([rel], [score]) for rel, score in zip(true_relevance[:k], scores[:k])])
    
    # DCG@5
    dcg_5 = ndcg_score([true_relevance], [scores], k=k)
    
    # nDCG@5
    ndcg_5 = ndcg_score([true_relevance], [scores], k=k, ignore_ties=True)
    
    # MRR
    try:
        rank = np.where(true_relevance[:k] == 1)[0][0] + 1
        mrr = 1.0 / rank
    except IndexError:
        mrr = 0.0
    
    return {
        'AP@5': ap_5,
        'MAP@5': map_5,
        'DCG@5': dcg_5,
        'nDCG@5': ndcg_5,
        'MRR': mrr
    }

def get_true_relevance(query, documents):
    query_words = re.findall(r'\w+', query.lower())
    relevance = []
    for content in documents:
        content_words = re.findall(r'\w+', content.lower())
        relevance.append(int(any(word in content_words for word in query_words)))
    return relevance

def compare_indices(query, documents, forward_index, inverted_index, tfidf_matrix, vectorizer):
    true_relevance = get_true_relevance(query, documents)
    
    # Прямой индекс
    forward_results = search_forward_index(query, forward_index)
    forward_scores = [1 if doc_id in forward_results[:5] else 0 for doc_id in range(len(documents))]
    
    # Обратный индекс
    inverted_results = search_inverted_index(query, inverted_index)
    inverted_scores = [1 if doc_id in inverted_results[:5] else 0 for doc_id in range(len(documents))]
    
    # TF-IDF индекс
    tfidf_results, tfidf_similarities = search_tfidf_index(query, tfidf_matrix, vectorizer)
    tfidf_scores = [1 if doc_id in tfidf_results[:5] else 0 for doc_id in range(len(documents))]
    
    # Расчет метрик
    metrics_forward = calculate_metrics(true_relevance, forward_scores)
    metrics_inverted = calculate_metrics(true_relevance, inverted_scores)
    metrics_tfidf = calculate_metrics(true_relevance, tfidf_scores)
    
    return {
        'Forward Index': metrics_forward,
        'Inverted Index': metrics_inverted,
        'TF-IDF Index': metrics_tfidf
    }

query = "сосо павлиашвили"
results = compare_indices(query, documents, forward_index, inverted_index, tfidf_matrix, vectorizer)

# Вывод результатов в виде таблицы
df = pd.DataFrame(results)
print(df)


        Forward Index  Inverted Index  TF-IDF Index
AP@5        -0.000000            -0.0     -0.000000
MAP@5        0.000000             0.0      0.000000
DCG@5        0.005842             1.0      0.830189
nDCG@5       0.000000             1.0      0.852928
MRR          0.000000             0.0      0.000000


/Users/vlad-zharkov/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/vlad-zharkov/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/vlad-zharkov/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/vlad-zharkov/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/Users/vlad-zharkov/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all threshold